In [1]:
import tensorflow as tf
import pathlib
import imageio
#tf.enable_eager_execution()
import os
import pandas as pd

path=os.getcwd()+'\\\sp-society-camera-model-identification'
os.chdir(path)


In [2]:
print(tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


In [3]:
trainpath = path+"\\train"
os.chdir(trainpath)

df1 = pd.read_pickle('train_images1.pkl')
df1.head(5)


,location,image
0,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.47058824, shape=(), dtype=float..."
1,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.6745098, shape=(), dtype=float3..."
2,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.59607846, shape=(), dtype=float..."
3,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.41568628, shape=(), dtype=float..."
4,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.5764706, shape=(), dtype=float3..."


In [4]:
df1.shape

(1375, 2)

In [5]:
df2 = pd.read_pickle('train_images2.pkl')
df2.shape

(1375, 2)

In [6]:

df1 = pd.concat([df1,df2])
df1.shape

(2750, 2)

In [7]:
df1['camera'] = df1['location'].apply(lambda x: x.split('\\')[-2])
df1.head()    

,location,image,camera
0,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.47058824, shape=(), dtype=float...",HTC-1-M7
1,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.6745098, shape=(), dtype=float3...",HTC-1-M7
2,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.59607846, shape=(), dtype=float...",HTC-1-M7
3,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.41568628, shape=(), dtype=float...",HTC-1-M7
4,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.5764706, shape=(), dtype=float3...",HTC-1-M7


In [8]:

cameras = os.listdir()
cameras.remove('.gitignore')
cameras.remove('train_images1.pkl')
cameras.remove('train_images2.pkl')
cameras.remove('model_weights.h5')
filecount = [len(os.listdir(camera))  for camera in cameras]
cam = pd.DataFrame({'camera_type':cameras,'filecount':filecount})

class_dict = {}
for id in cam.index:
    class_dict.update({cam.loc[id,'camera_type']:id})
    
class_dict

{'HTC-1-M7': 0,
 'iPhone-4s': 1,
 'iPhone-6': 2,
 'LG-Nexus-5x': 3,
 'Motorola-Droid-Maxx': 4,
 'Motorola-Nexus-6': 5,
 'Motorola-X': 6,
 'Samsung-Galaxy-Note3': 7,
 'Samsung-Galaxy-S4': 8,
 'Sony-NEX-7': 9}

In [9]:
df1['class'] = df1['camera'].apply(lambda x:class_dict[x])
df1.head()

,location,image,camera,class
0,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.47058824, shape=(), dtype=float...",HTC-1-M7,0
1,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.6745098, shape=(), dtype=float3...",HTC-1-M7,0
2,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.59607846, shape=(), dtype=float...",HTC-1-M7,0
3,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.41568628, shape=(), dtype=float...",HTC-1-M7,0
4,D:\Study\Capstone\cameraDetection\sp-society-c...,"(((tf.Tensor(0.5764706, shape=(), dtype=float3...",HTC-1-M7,0


In [10]:
df1['image_array'] = df1['image'].apply(lambda x:x.numpy())

In [11]:
#getting training and test datasets
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(df1['image_array'], df1['class'].values,test_size=.33, random_state=42)


In [12]:
del df1

In [13]:


import numpy as np
def reshape_input(Xtrain):
    Xtrain = np.stack(Xtrain.values) #convert a pandas series of arrays into one single array
    input_cols = Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3]
    Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],input_cols)) #flatten the data to single set of neurons
    return Xtrain

In [14]:
Xtrain= reshape_input(Xtrain)

In [15]:
#categorize output data. 
import keras
from keras.utils.np_utils import to_categorical

def reshape_target(Ytrain):
    Ytrain = to_categorical(Ytrain,num_classes=10)
    return Ytrain

Ytrain = reshape_target(Ytrain)

Using TensorFlow backend.


In [16]:
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD

In [17]:
from keras.callbacks import EarlyStopping

In [18]:
def get_model(Xtrain,Ytrain,output_units=500):
    model = Sequential()
    input_cols = Xtrain.shape[1]
    model.add(Dense(output_units, activation='relu',input_shape=(input_cols,)))
    model.add(Dense(output_units))
    model.add(Dense(output_units))
    model.add(Dense(output_units))
    model.add(Dense(500))
    model.add(Dense(500))
    model.add(Dense(10,activation='softmax'))
    return model

In [19]:
#Testing a normal neural network

earlystopping_monitor = EarlyStopping(monitor='val_accurary',min_delta=.1,patience=10,verbose=1)
lr = [0.001]
#changing the lr from .001 to .0001 to increase accuracy more than 32
for l in lr:
    model = get_model(Xtrain,Ytrain,1000)
    model.load_weights('model_weights.h5')
    sgd_optimizer = SGD(l)
    model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(Xtrain,Ytrain,validation_split=.3,epochs=100,callbacks=[earlystopping_monitor])
model.save_weights('model_weights.h5')
#using lr of .001 has the best improvement in accuracy so far(.17 to .48 over 10 epoches) So, going lesser than that and increaseing the epoches to 25
#.00001 is slow as it increases accuracy from.10 to .27 in 25 epoches
#.0001 .13->.46 in 25 epoches (caused resouorceexhaustedError)
#.001 .17 -> .72 (it fluctuates after .71, maxes at .72 and decreases again). 
#Increasing the output units to 1000 to see the increase, with lr of .001. Accuracy reaches upto .76 but its not converging. 
#WOULD REDUCING LEARNING RATE AFTER 20 EPOCHES HELP TO CONVEREGE?
#validation accuracy is still at .28 max

#to use Model checkpoint, refer link here: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
#preprocessing and training
#training->hyperparameter
#preprocessing -> rotate imags and others

Train on 1289 samples, validate on 553 samples
Epoch 1/100
1289/1289 [==============================] - 4s 3ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 2.5107 - val_accuracy: 0.3237
Epoch 2/100
  64/1289 [>.............................] - ETA: 2s - loss: 0.0097 - accuracy: 1.0000

C:\Users\andyz\Anaconda3\envs\tf_gpu\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_accurary` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1289/1289 [==============================] - 3s 3ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 2.4735 - val_accuracy: 0.3237
Epoch 3/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0103 - accuracy: 1.0000 - val_loss: 2.5109 - val_accuracy: 0.3309
Epoch 4/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 2.5131 - val_accuracy: 0.3291
Epoch 5/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 2.5295 - val_accuracy: 0.3201
Epoch 6/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 2.5305 - val_accuracy: 0.3291
Epoch 7/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 2.4996 - val_accuracy: 0.3291
Epoch 8/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 2.5107 - val_accuracy: 0.32

1289/1289 [==============================] - 3s 3ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 2.6464 - val_accuracy: 0.3183
Epoch 59/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 2.6456 - val_accuracy: 0.3201
Epoch 60/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 2.6467 - val_accuracy: 0.3201
Epoch 61/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 2.6510 - val_accuracy: 0.3219
Epoch 62/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 2.6461 - val_accuracy: 0.3219
Epoch 63/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 2.6443 - val_accuracy: 0.3201
Epoch 64/100
1289/1289 [==============================] - 3s 3ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 2.6463 - val_accuracy

In [20]:
import os, psutil
def usage():
    process = psutil.Process(os.getpid())
    return process.memory_info()[0] / float(2 ** 20)
usage()

3307.9921875

In [21]:
#from tensorflow.contrib.memory_stats import BytesInUse
#with tensorflow.device('/device:GPU:0'):  # Replace with device you are interested in
  #bytes_in_use = BytesInUse()
#print(bytes_in_use)